In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline

import os, pickle, re, glob, time
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from scipy import stats
from collections import Counter

sns.set_style('ticks')
pd.set_option('precision', 2)
#np.set_printoptions('precision', 2)

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from lib import utils

In [4]:
from Bio.Blast import NCBIXML

args = pickle.load(open('arguments.pickle', 'rb'))
align_dir = os.path.join(args.genome_dir, 'align')

# read alignment

In [5]:
alignment_list = []

for name in glob.glob(os.path.join(align_dir, 'alignment*.xml')):
    alignment_list += [name]

In [6]:
len(alignment_list)

7

In [7]:
snps_result = []
start_end_result = []

for name in alignment_list:
    result, start_end = utils.start_end_list_func(name)
    snps_result += result
    start_end_result += start_end


In [8]:
#start_end ของทุก alignment (1 sample อาจมีหลายค่า)
start_end_df = pd.DataFrame(start_end_result, columns=['id', 'start_end', 'length'])

In [9]:
id_list = sorted(list(set(start_end_df['id'])))
len(id_list)

3406

In [10]:
start_end_df = start_end_df.sort_values(['id'])
start_end_df.head()

,id,start_end,length
242,1034260,18_29891,29874
292,1034261,5_29803,29799
408,1034262,50_29744,29695
261,1034263,61_29744,29684
32,1034264,3_29891,29889


# delete insertion 

In [11]:
snps_df = pd.DataFrame(snps_result, columns=['id','query','position','sbjct', 'start_end', 'length'])
snps_df = snps_df.sort_values(['id','length','position'])

In [12]:
#save before delete insertion
snps_df.to_csv(os.path.join(args.preprocess_dir, 'snps_insertion_df.csv'), index=False)

In [13]:
#id ที่จะ delete
id_del = sorted(list(set(snps_df[snps_df['query']=='-']['id'])))
len(id_del)

11

In [14]:
#check ว่่าควรจะเหลือข้อมูลกี่แถว
snps_df.shape[0] -  snps_df[(snps_df['query']=='-')].shape[0]

114001

In [15]:
#delete insertion
snps_df = utils.del_insertion(id_del, snps_df)

In [16]:
snps_df.shape

(114001, 6)

In [17]:
snps_df['change']=snps_df['query']+snps_df['position'].astype(str)+snps_df['sbjct']
snps_df = snps_df.sort_values(['id'])
snps_df.head()

,id,query,position,sbjct,start_end,length,change
6549,1034260,C,241,T,18_29891,29874,C241T
6610,1034260,C,27286,N,18_29891,29874,C27286N
6609,1034260,T,27285,N,18_29891,29874,T27285N
6608,1034260,A,27284,N,18_29891,29874,A27284N
6607,1034260,A,27283,N,18_29891,29874,A27283N


In [19]:
snps_df.to_csv(os.path.join(args.preprocess_dir, 'snps_df.csv'), index=False)

In [20]:
start_end_df.to_csv(os.path.join(args.preprocess_dir, 'startstop_df.csv'), index=False)

In [21]:
#check ว่าไม่มีแล้วจริงๆ
snps_df[snps_df['query']=='-']

,id,query,position,sbjct,start_end,length,change
